In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
import pandas as pd

spark = SparkSession.builder.appName("stream").getOrCreate()  # type: ignore

employment_src_path = "data/employment.csv"

In [ ]:
df = (
    spark.read.csv(
        employment_src_path,
        header=True,
        inferSchema=True,
    )
    .withColumn("period_year", f.year(f.to_date("Period", "yyyy.MM")))
    .filter(f.col("period_year").isin([2013, 2014]))
    .groupBy(f.col("period_year"))
    .agg(f.sum("Data_value").alias("employed"))
)

df.show()

In [31]:
employment = spark.read.csv(
    employment_src_path,
    header=True,
    inferSchema=True,
)
employment_with_year = employment.withColumn(
    "period_year", f.year(f.to_date("Period", "yyyy.MM"))
)
employment_with_year_filtered = employment_with_year.filter(
    f.col("period_year").isin([2013, 2014])
)
employment_per_year = employment_with_year_filtered.groupBy(f.col("period_year")).agg(
    f.sum("Data_value").alias("employed")
)

employment_per_year.show()

+-----------+-------------------+
|period_year|           employed|
+-----------+-------------------+
|       2013|7.898903575750296E7|
|       2014|8.104320780059406E7|
+-----------+-------------------+



In [29]:
df = pd.read_csv(employment_src_path)
df["period_year"] = df["Period"].astype(str).str.split(".").str[0].astype(int)
df = df[df["period_year"].isin([2013, 2014])]
df = df.groupby("period_year").agg({"Data_value": "sum"}).reset_index()
df.columns = ["period_year", "employed"]

df.head()

   period_year      employed
0         2013  7.898904e+07
1         2014  8.104321e+07
